# Import netcdf files

In [1]:
from netCDF4 import Dataset
#import pandas as pd
import numpy as np
#import numpy.ma as ma
import vtk
from vtk.util import numpy_support
from pyproj import Transformer, transform
#from vtk.numpy_interface import dataset_adapter as dsa

In [2]:
src_nc = Dataset("../02_data/swc.nc", mode = "r", format = "NETCDF4")

In [3]:
print(src_nc.data_model)

NETCDF4


In [4]:
print(src_nc.groups)

OrderedDict()


In [5]:
print(src_nc.dimensions)

OrderedDict([('x', <class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 175
), ('y', <class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 225
), ('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 1318
)])


In [6]:
print(src_nc.variables)

OrderedDict([('lon', <class 'netCDF4._netCDF4.Variable'>
float64 lon(y, x)
    standard_name: longitude
    long_name: longitude
    units: degrees_east
    _CoordinateAxisType: Lon
unlimited dimensions: 
current shape = (225, 175)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('lat', <class 'netCDF4._netCDF4.Variable'>
float64 lat(y, x)
    standard_name: latitude
    long_name: latitude
    units: degrees_north
    _CoordinateAxisType: Lat
unlimited dimensions: 
current shape = (225, 175)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('time', <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    long_name: time
    units: hours since 2016-1-1 00:00:00
    calendar: standard
    axis: T
unlimited dimensions: time
current shape = (1318,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('SWC_L01', <class 'netCDF4._netCDF4.Variable'>
float64 SWC_L01(time, y, x)
    long_name: soil water content of soil layer

In [7]:
print(src_nc.ncattrs)

<built-in method ncattrs of netCDF4._netCDF4.Dataset object at 0x7feb725d2228>


In [8]:
# access data of src_nc
# src.variables["var_name"][time][y][x]
print(len(src_nc.variables["SWC_L01"][0][:][1]))

175


/home/boog/software/anaconda3/envs/py3env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  This is separate from the ipykernel package so we can avoid doing imports until


## Get variable data

### Get coordinates 

* this netcdf file uses coordinates latitude and longitude
* both coordiantes depend on the dimensions x and y
* x (175 elements) and y (225 elements) create a grid of 175*225 elements
* each element has a value for one of the variables lat, lon, scw_l01, scw_l02
* get lat and lon out

In [9]:
lat_dat = src_nc.variables["lat"][:]

# this procuces a masked array, --> use .filled()
lat_dat = lat_dat.filled()
lat_dat.shape[0]

225

In [10]:
lon_dat = src_nc.variables["lon"][:]

# this procuces a masked array, --> use .filled()
lon_dat = lon_dat.filled()
lon_dat.shape

(225, 175)

### Get time

In [11]:
time = src_nc.variables["time"]
time

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    long_name: time
    units: hours since 2016-1-1 00:00:00
    calendar: standard
    axis: T
unlimited dimensions: time
current shape = (1318,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [12]:
time.units

'hours since 2016-1-1 00:00:00'

In [13]:
# get time data
time = time[:]

### Get swc data

In [14]:
# get swc_l01
swcl01 = src_nc.variables["SWC_L01"]
swcl02 = src_nc.variables["SWC_L02"]

In [15]:
# get data from variable
# swcl01[:] will produce a masked arraya
# use .filled() to add the fill values in the array
swcl01_dat = swcl01[:].filled()
swcl02_dat = swcl02[:].filled()
type(swcl01_dat)

/home/boog/software/anaconda3/envs/py3env/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  after removing the cwd from sys.path.
/home/boog/software/anaconda3/envs/py3env/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  """


numpy.ndarray

## Transfrom to vtkPolyData

The type vtkPolydata can use the cell coordinates of the NetCDF file as points and vertice cells. Each point represents, therefore, a cells as well.

### Def vtkPolydata

In [16]:
# def point array
points = np.array([lon_dat.flatten(), 
                   lat_dat.flatten(),
                   np.repeat(0, len(lat_dat.flatten()))
                ]).transpose()
len(points)

39375

In [17]:
# def and fill vtkPoints
src_points = vtk.vtkPoints()
src_cells = vtk.vtkCellArray()
#src_points.SetNumberOfPoints(len(points))

# define single point object
#p = np.empty(3, dtype=np.float64)
#p[2] = 0

# def coordinate transformer
transf = Transformer.from_crs(
                        "EPSG:4326",
                        "EPSG:5684",
                        always_xy=True)

for i in range(len(points)):
    p = transf.transform(points[i][0], points[i][1])
    ind = src_points.InsertNextPoint(p[0], p[1], 0)
    src_cells.InsertNextCell(1)
    src_cells.InsertCellPoint(ind)

In [18]:
src_points.GetNumberOfPoints()

39375

In [19]:
#cells = np.array([[1]*src_points.GetNumberOfPoints(),
#                 range(src_points.GetNumberOfPoints())
#                 ]).transpose()
#cells = np.ascontiguousarray(cells, dtype=np.int64)
#cells
#src_cells = vtk.vtkCellArray()
#src_cells.SetCells(src_points.GetNumberOfPoints(),
#                   numpy_support.numpy_to_vtkIdTypeArray(cells, deep=True))  

In [20]:
# create vtkPolydata
src_poly = vtk.vtkPolyData()
# be aware to set directions as x-y --> lon-lat not lat-lon
#src_poly.SetDimensions(lat_dat.shape[1], lat_dat.shape[0],1)
src_poly.SetPoints(src_points)

# define vertice based cells
src_poly.SetVerts(src_cells)

### Add data

In [21]:
def add_data_to_src_poly(src_poly, time, swcl01_dat, swcl02_dat):
    
    for i in range(len(time)):
        arr_name1 = "SWC_L01_%s" % str(int(time[i]))
        arr_name2 = "SWC_L02_%s" % str(int(time[i]))
        new_point_arr_vtk1 = numpy_support.numpy_to_vtk(swcl01_dat[i].flatten())
        new_point_arr_vtk1.SetName(arr_name1)
        new_point_arr_vtk2 = numpy_support.numpy_to_vtk(swcl02_dat[i].flatten())
        new_point_arr_vtk2.SetName(arr_name2)
        src_poly.GetPointData().AddArray(new_point_arr_vtk1)
        src_poly.GetPointData().AddArray(new_point_arr_vtk2)
        src_poly.Modified()

In [22]:
add_data_to_src_poly(src_poly, time, swcl01_dat, swcl02_dat)

In [23]:
src_poly.GetPointData().GetArray(1317)

(vtkCommonCorePython.vtkDoubleArray)0x7feb725c6dc8

#### Write test file

In [24]:
def write_netcdf_as_vtp(src_obj, outputfile_name):
    write_ouput = vtk.vtkXMLPolyDataWriter()
    write_ouput.SetInputData(src_obj)
    write_ouput.SetFileName(outputfile_name)
    write_ouput.Write() 

In [25]:
#write_netcdf_as_vtp(src_poly, "dummy_allts.vtp")

## Map data on OGS-VTU

### Read OGS-VTU

In [26]:
def read_ogs_vtu(in_filepath):
    dst = vtk.vtkXMLUnstructuredGridReader()
    dst.SetFileName(in_filepath)
    dst.Update()
    return(dst.GetOutput())

In [27]:
ogs_src_vtu = read_ogs_vtu("../02_data/Selke_3D_Top.vtu")
ogs_src_vtu

(vtkCommonDataModelPython.vtkUnstructuredGrid)0x7feb704b5048

### Interpolate data on OGS-mesh

In [28]:
def int_kernel():
    # choose your interpolation kernel
    
    # gaussian kernel
    #int_kernel = vtk.vtkGaussianKernel()
    #int_kernel.SetSharpness(2)
    #int_kernel.SetRadius(4000)
    
    #voronoi -- good for categorial data
    int_kernel = vtk.vtkVoronoiKernel()
    
    # shepard kernel
    #int_kernel = vtk.vtkShepardKernel()
    #int_kernel.SetPowerParameter(2)
    #int_kernel.SetRadius(4000)

    return int_kernel


In [29]:
def map_data_on_ogs_vtu(src_poly, dst):
    interpolator = vtk.vtkPointInterpolator()
    interpolator.SetInputData(dst)
    interpolator.SetSourceData(src_poly)
    # def interpolation algorithm
    interpolator.SetKernel(int_kernel())
    # def value if interpolation does not work
    interpolator.SetNullValue(-9999)
    interpolator.Update()
    return(interpolator.GetOutput())

In [30]:
out_vtu = map_data_on_ogs_vtu(src_poly, ogs_src_vtu)
out_vtu

(vtkCommonDataModelPython.vtkUnstructuredGrid)0x7feb704b50a8


### Output interpolated data

In [32]:
def write_mapped_ogs_vtu(out_vtu, out_filename):
    write_output = vtk.vtkXMLUnstructuredGridWriter()
    write_output.SetFileName(out_filename)
    write_output.SetInputData(out_vtu)
    write_output.Write()

In [33]:
write_mapped_ogs_vtu(out_vtu, "map_netcdf_on_vtu2.vtu")